In [27]:
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

In [28]:
def get_date_rage(release):
    year, month, day = release.split('-')
    release_3ago = (datetime(int(year), int(month), int(day)) - relativedelta(years=3)).strftime('%Y-%m-%d')
    return [release_3ago, release]

In [29]:
actor_movie = pd.read_csv('./actor_movie.csv', index_col=0)
actor_movie

,actor_code,movie_name,movie_code
0,20224649,공조2:인터내셔날,20215601
1,20224649,해피뉴이어(가제),20217807
2,20224649,기적(Miracle),20200474
3,20224649,엑시트(EXIT),20184621
4,20224649,공조(Confidential Assignment),20151228
...,...,...,...
2890,10054008,가위(A nightmare),20000009
2891,10054008,동감(Ditto),20000011
2892,10054008,MOB 2025 1편 : 전운(MOB 2025),20008019
2893,10054008,주유소 습격사건(Attack The Gas Station!),19990067


In [30]:
movie_actor_df = pd.read_csv('movie_actor_df.csv', index_col=0)
movie_actor_df = movie_actor_df.fillna(0)
movie_actor_df = movie_actor_df.astype({'actor2_code': 'int'})
movie_actor_df

,movie_name,code,actor1_code,actor2_code
0,극한직업,20182530,10019065,20125825
1,어벤져스: 엔드게임,20184889,10016538,10077150
2,겨울왕국 2,20197803,10077984,10055204
3,알라딘,20183867,10044943,10052967
4,기생충,20183782,10037018,10055970
...,...,...,...,...
89,프리즌 이스케이프,20207893,10009288,10056554
90,검객,20178401,10060612,20129449
91,조제,20201002,10087751,20279929
92,사라진 시간,20193859,10067353,10030161


In [82]:
movie_df = pd.read_excel('../Movie_Data.xlsx').loc[:,['code','release']]

merge_movie_info_df = pd.merge(left = movie_actor_df , right = movie_df, how = "left", on = "code")
merge_movie_info_df['actor_power'] = np.nan
merge_movie_info_df=merge_movie_info_df.astype({'release': 'str'})
merge_movie_info_df

,movie_name,code,actor1_code,actor2_code,release,actor_power
0,극한직업,20182530,10019065,20125825,2019-01-23,NaN
1,어벤져스: 엔드게임,20184889,10016538,10077150,2019-04-24,NaN
2,겨울왕국 2,20197803,10077984,10055204,2019-11-21,NaN
3,알라딘,20183867,10044943,10052967,2019-05-23,NaN
4,기생충,20183782,10037018,10055970,2019-05-30,NaN
...,...,...,...,...,...,...
89,프리즌 이스케이프,20207893,10009288,10056554,2020-05-06,NaN
90,검객,20178401,10060612,20129449,2020-09-23,NaN
91,조제,20201002,10087751,20279929,2020-12-10,NaN
92,사라진 시간,20193859,10067353,10030161,2020-06-18,NaN


In [32]:
movie_info_total = pd.read_csv('movie_info_total.csv', index_col=0)

In [33]:
merge_movie_df = pd.merge(left = actor_movie , right = movie_info_total, how = "left", on = "movie_code")

In [34]:
drop_index = merge_movie_df[merge_movie_df['release'] == '해당정보없음'].index
merge_movie_df = merge_movie_df.drop(drop_index)
merge_movie_df=merge_movie_df.reset_index(drop=True)
merge_movie_df

,actor_code,movie_name,movie_code,release,acc_audience_num
0,20224649,기적(Miracle),20200474,2021-09-15,487617
1,20224649,엑시트(EXIT),20184621,2019-07-31,9426051
2,20224649,공조(Confidential Assignment),20151228,2017-01-18,7817446
3,10021341,이터널스(Eternals),20218256,2021-11-03,2888988
4,10021341,백두산(ASHFALL),20192206,2019-12-19,8252669
...,...,...,...,...,...
2331,10054008,봄날은 간다(One Fine Spring Day),20010013,2001-09-28,0
2332,10054008,가위(A nightmare),20000009,2000-07-29,0
2333,10054008,동감(Ditto),20000011,2000-05-27,0
2334,10054008,주유소 습격사건(Attack The Gas Station!),19990067,1999-10-02,2310000


In [83]:
for i in range(len(merge_movie_info_df)):
    movie_code = merge_movie_info_df['code'][i]
    actor1_code = merge_movie_info_df['actor1_code'][i]
    actor2_code = merge_movie_info_df['actor2_code'][i]
    release = list(merge_movie_info_df.loc[merge_movie_info_df['code']==movie_code, 'release'].values)[0]

    date_range = get_date_rage(release)
    filtered_movie_list = merge_movie_df[
        ((merge_movie_df['actor_code'] == actor1_code) |
        (merge_movie_df['actor_code'] == actor2_code)) &
        ((merge_movie_df['release'] >= date_range[0]) & (merge_movie_df['release'] <= date_range[1]))
    ]

    audience_mean = np.mean(filtered_movie_list['acc_audience_num'])
    merge_movie_info_df.loc[merge_movie_info_df['code']==movie_code, 'actor_power'] = audience_mean

    

In [84]:
merge_movie_info_df

,movie_name,code,actor1_code,actor2_code,release,actor_power
0,극한직업,20182530,10019065,20125825,2019-01-23,4.539712e+06
1,어벤져스: 엔드게임,20184889,10016538,10077150,2019-04-24,9.367612e+06
2,겨울왕국 2,20197803,10077984,10055204,2019-11-21,9.751492e+06
3,알라딘,20183867,10044943,10052967,2019-05-23,6.752496e+06
4,기생충,20183782,10037018,10055970,2019-05-30,5.109347e+06
...,...,...,...,...,...,...
89,프리즌 이스케이프,20207893,10009288,10056554,2020-05-06,8.232217e+04
90,검객,20178401,10060612,20129449,2020-09-23,1.902370e+05
91,조제,20201002,10087751,20279929,2020-12-10,1.335327e+06
92,사라진 시간,20193859,10067353,10030161,2020-06-18,1.872178e+06


In [85]:
merge_movie_info_df.to_csv('movie_actor_power.csv', encoding='utf-8-sig')